In [4]:
import netCDF4 as nc
import numpy as np
from netCDF4 import Dataset

from os import listdir
import dateutil.parser  #引入这个库来将日期字符串转成统一的datatime时间格式

import json

# 记录城市对应经纬度信息
cityDict={'深圳':[],'上海':[],'广州':[],'北京':[],'成都':[],
          '杭州':[],'重庆':[],'西安':[],'苏州':[],'武汉':[],
          '南京':[],'天津':[],'郑州':[],'长沙':[],'东莞':[],
          '佛山':[],'宁波':[],'青岛':[],'沈阳':[],'合肥':[]}
with open("/Users/rumeng/Downloads/qingzang/test/city.txt", "r") as f:
    for line in f.readlines():
        line = line.strip('\n')
        for i in cityDict.keys():
            if i in line:
                pos=line.split()[0]
                cityDict[i].append(pos)
# 记录城市对应的lon和lat维的索引i j                
ijDict={'深圳':[],'上海':[],'广州':[],'北京':[],'成都':[],
          '杭州':[],'重庆':[],'西安':[],'苏州':[],'武汉':[],
          '南京':[],'天津':[],'郑州':[],'长沙':[],'东莞':[],
          '佛山':[],'宁波':[],'青岛':[],'沈阳':[],'合肥':[]}
with open("/Users/rumeng/Downloads/qingzang/test/ij2city.txt", "r") as f:
    for line in f.readlines():
        line = line.strip('\n').split()
        ijDict[line[2]].append([int(line[0]),int(line[1])])
        



In [5]:
# 获得每月的天数
def getDays(year,month):
#     year=int(dateStr[:4])
#     month=int(dateStr[4:])
    if month in [1,3,5,7,8,10,12]:
        return 31
    elif month in [4,6,9,11]:
        return 30
    else:
        if year%4!=0:
            return 28
        elif year%100==0 and year%400!=0:
            return 28
        else:
            return 29

In [27]:
path='/Users/rumeng/Downloads/qingzang/forcing_03hr'
labelpath='/Users/rumeng/Downloads/qingzang/forcing_03hr/'
# path='/Users/rumeng/Downloads'
# labelpath='/Users/rumeng/Downloads/'
# /Users/rumeng/Downloads/temp_ITPCAS-CMFD_V0106_B-01_03hr_010deg_201306.nc

# filelist = listdir(path)

data=[]
cityDate={}
objPath='/Users/rumeng/Downloads/qingzang/result/'
yearInfo=['2014','2015','2016','2017','2018']
# yearInfo=['2000','2001','2002','2003','2004','2005','2006',
#           '2007','2008','2009','2010','2011','2012','2013',
#           '2014','2015','2016','2017','2018']
# monthInfo=['07','08','09','10','11','12']
monthInfo=['01','02','03','04','05','06','07','08','09','10','11','12']
fileName='temp_CMFD_V0106_B-01_03hr_010deg_'
# fileName='temp_ITPCAS-CMFD_V0106_B-01_03hr_010deg_'
for y in yearInfo:
    for m in monthInfo:
        year=int(y)
        month=int(m)
# for file in filelist:
#     dateStr=file[-9:-3]
#     year=int(dateStr[:4])
#     month=int(dateStr[4:])
        days=getDays(year,month)
        print(year,month,days)

        data=[]
        cityDate={}

        ncTemp=Dataset(labelpath+fileName+y+m+'.nc')
        time = nc.num2date(ncTemp.variables['time'][:],'hours since 1900-1-1 00:00:00').data

        all_times = ncTemp.variables['time']
        sdt = dateutil.parser.parse(str(year)+'-'+str(month)+"-1T00:00:00")#"2000-1-1T00:00:00"
        edt = dateutil.parser.parse(str(year)+'-'+str(month)+'-'+str(days)+"T21:00:00")#"2000-1-31T21:00:00"
        st_idx = nc.date2index(sdt, all_times)
        et_idx = nc.date2index(edt, all_times)
#     print(sdt,edt)
#     print(st_idx,et_idx)
        temp = ncTemp.variables['temp'][st_idx:et_idx+1,:,:].data
    
        for city in cityDict.keys():
    #         print(city)
            for i in ijDict[city]:
                for j in range(days*8):
            #         print(temp[j][i[0]][i[1]])
                    if temp[j][i[0]][i[1]]==-32767:
                        continue
                    year=time[j].year
                    month=time[j].month
                    day=time[j].day  
                    thisDate=str(year)+'-'+str(month)+'-'+str(day)
                    thisTemp=temp[j][i[0]][i[1]]
                    if city+thisDate not in cityDate.keys():
                        cityDate[city+thisDate]=len(data)
                        data.append({"city":city,"date":thisDate,"temp":{"minTemp":thisTemp,"maxTemp":thisTemp}})
                    else:
                        if thisTemp>data[cityDate[city+thisDate]]["temp"]["maxTemp"]:
                            data[cityDate[city+thisDate]]["temp"]["maxTemp"]=thisTemp
                        if thisTemp<data[cityDate[city+thisDate]]["temp"]["minTemp"]:
                            data[cityDate[city+thisDate]]["temp"]["minTemp"]=thisTemp

        length=len(data)
        for i in range(length):
            data[i]["temp"]["maxTemp"]=str(data[i]["temp"]["maxTemp"])
            data[i]["temp"]["minTemp"]=str(data[i]["temp"]["minTemp"])
            city=data[i]["city"]
            j = json.dumps(data[i],ensure_ascii=False)        
          # print(j)  
            with open(objPath+city+'Temp.json', 'a',encoding='utf-8-sig') as json_file:
                json_file.write(j)
                json_file.write('\n') 

2014 1 31
2014 2 28
2014 3 31
2014 4 30
2014 5 31
2014 6 30
2014 7 31
2014 8 31
2014 9 30
2014 10 31
2014 11 30
2014 12 31
2015 1 31
2015 2 28
2015 3 31
2015 4 30
2015 5 31
2015 6 30
2015 7 31
2015 8 31
2015 9 30
2015 10 31
2015 11 30
2015 12 31
2016 1 31
2016 2 29
2016 3 31
2016 4 30
2016 5 31
2016 6 30
2016 7 31
2016 8 31
2016 9 30
2016 10 31
2016 11 30
2016 12 31
2017 1 31
2017 2 28
2017 3 31
2017 4 30
2017 5 31
2017 6 30
2017 7 31
2017 8 31
2017 9 30
2017 10 31
2017 11 30
2017 12 31
2018 1 31
2018 2 28
2018 3 31
2018 4 30
2018 5 31
2018 6 30
2018 7 31
2018 8 31
2018 9 30
2018 10 31
2018 11 30
2018 12 31


In [34]:
'''
import requests
key = 'GjG3XAdmywz7CyETWqHwIuEC6ZExY6QT'
for y in yearInfo:
    for m in monthInfo:
        print(y,m)
        ncTemp=Dataset(labelpath+fileName+y+m+'.nc')
        lon=ncTemp.variables['lon'][:]
        lat=ncTemp.variables['lat'][:]
        
        for city in cityDict.keys():
            for i in ijDict[city]:
                thisLon=lon[i[1]]
                thisLat=lat[i[0]]
                r = requests.get(url='http://api.map.baidu.com/geocoder/v2/', params={'location':str(thisLat)+','+str(thisLon),'ak':key,'output':'json'})
                result = r.json()
                thisprovince = result['result']['addressComponent']['province']
                thiscity = result['result']['addressComponent']['city']
                if city not in thisprovince+thiscity:
                    print(i,city,thisprovince+thiscity)
'''               

"\nimport requests\nkey = 'GjG3XAdmywz7CyETWqHwIuEC6ZExY6QT'\nfor y in yearInfo:\n    for m in monthInfo:\n        print(y,m)\n        ncTemp=Dataset(labelpath+fileName+y+m+'.nc')\n        lon=ncTemp.variables['lon'][:]\n        lat=ncTemp.variables['lat'][:]\n        \n        for city in cityDict.keys():\n            for i in ijDict[city]:\n                thisLon=lon[i[1]]\n                thisLat=lat[i[0]]\n                r = requests.get(url='http://api.map.baidu.com/geocoder/v2/', params={'location':str(thisLat)+','+str(thisLon),'ak':key,'output':'json'})\n                result = r.json()\n                thisprovince = result['result']['addressComponent']['province']\n                thiscity = result['result']['addressComponent']['city']\n                if city not in thisprovince+thiscity:\n                    print(i,city,thisprovince+thiscity)\n"

In [12]:
# 处理两日间温差 第二天的平均温度-第一天的平均温度
sourcePath='/Users/rumeng/Downloads/qingzang/trainSet/climate/'
objPath='/Users/rumeng/Downloads/qingzang/trainSet/climate2/'
for city in cityDict.keys():
    print(city)
    with open(sourcePath+city+'.json', 'r',encoding='utf-8-sig') as f:
        lines = f.readlines()
        wencha = 0
        pastAvgTemp =0
        for line in lines:
            climate=json.loads(line)
            avgTemp = (float(climate['temp']['maxTemp'])+float(climate['temp']['minTemp']))/2
            if pastAvgTemp==0:
                wencha=0
            else:
                wencha = avgTemp-pastAvgTemp
            pastAvgTemp = avgTemp
            climate['tempDiff'] = str(round(wencha,2))
            j = json.dumps(climate,ensure_ascii=False)     
            with open(objPath+city+'.json', 'a',encoding='utf-8-sig') as json_file:
                json_file.write(j)
                json_file.write('\n') 

深圳
上海
广州
北京
成都
杭州
重庆
西安
苏州
武汉
南京
天津
郑州
长沙
东莞
佛山
宁波
青岛
沈阳
合肥


In [8]:
wencha1=10
wencha2=-11
wencha=wencha1 if abs(wencha1)>abs(wencha2) else wencha2
print(wencha)

-11
